<a href="https://colab.research.google.com/github/onenechan/BSch3v_library/blob/master/JAXAday4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
pip install healpy

In [37]:
import urllib.request
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [38]:
url = 'https://data.darts.isas.jaxa.jp/pub/pds3/sln-l-tc-5-sldem2013-v1.0/lon000/data/DTM_MAP_01_N00E000S01E001SC.lbl'
norder = 10 #nside1024
nside = 2 ** norder
#cnt_pix = nside ** 2 * 12
print(nside)

1024


In [39]:
def lbl2dict(lbl_url):
  import urllib.request  #URL読み出し用
  import re  #正規表現用

  dictionary = {}

  # urlの読み出し(byte型)
  com=urllib.request.urlopen(lbl_url)
  byte_data=com.read()
  com.close()

  # str型に変換
  string_data=byte_data.decode("utf-8")

  # 改行で分けてlistに代入
  a=string_data.split('\r\n')

  # 数字と小数点、正負の部分を抽出(str型)
  _UPPER_LEFT_LATITUDE = re.search(r'-?[\d\.]+', a[24])
  _UPPER_LEFT_LONGITUDE = re.search(r'-?[\d\.]+', a[25])
  _UPPER_RIGHT_LATITUDE = re.search(r'-?[\d\.]+', a[26])
  _UPPER_RIGHT_LONGITUDE = re.search(r'-?[\d\.]+', a[27])
  _LOWER_LEFT_LATITUDE =  re.search(r'-?[\d\.]+', a[28])
  _LOWER_LEFT_LONGITUDE = re.search(r'-?[\d\.]+', a[29])
  _LOWER_RIGHT_LATITUDE =  re.search(r'-?[\d\.]+', a[30])
  _LOWER_RIGHT_LONGITUDE = re.search(r'-?[\d\.]+', a[31])

  _LINE= re.search(r'-?[\d\.]+', a[75])
  _LINESAMPLE = re.search(r'-?[\d\.]+', a[76])

  # floatで保管
  dictionary["UPPER_LEFT_LATITUDE"]=float(_UPPER_LEFT_LATITUDE.group())
  dictionary["UPPER_LEFT_LONGITUDE"]=float(_UPPER_LEFT_LONGITUDE.group())
  dictionary["UPPER_RIGHT_LATITUDE"]=float(_UPPER_RIGHT_LATITUDE.group())
  dictionary["UPPER_RIGHT_LONGITUDE"]=float(_UPPER_RIGHT_LONGITUDE.group())
  dictionary["LOWER_LEFT_LATITUDE"]=float(_LOWER_LEFT_LATITUDE.group())
  dictionary["LOWER_LEFT_LONGITUDE"]=float(_LOWER_LEFT_LONGITUDE.group())
  dictionary["LOWER_RIGHT_LATITUDE"]=float(_LOWER_RIGHT_LATITUDE.group())
  dictionary["LOWER_RIGHT_LONGITUDE"]=float(_LOWER_RIGHT_LONGITUDE.group())
  dictionary["LINE"]= int(_LINE.group())
  dictionary["LINESAMPLE"]= int(_LINESAMPLE.group())

  return dictionary

In [40]:
def readdics(lbl_url,nside):
  #dictionaryから呼び出す
  import numpy as np
  import healpy as hp

  dictionary = lbl2dict(url)

  #緯度経度の羅列()
  lon2pix=np.linspace(dictionary["LOWER_LEFT_LONGITUDE"],dictionary["LOWER_RIGHT_LONGITUDE"],dictionary["LINE"])
  lat2pix=np.linspace(dictionary["UPPER_LEFT_LATITUDE"],dictionary["LOWER_LEFT_LATITUDE"],dictionary["LINE"])
  #羅列したものをgrid状にしてる
  lon2pix2,lat2pix2=np.meshgrid(lon2pix,lat2pix)
  #print(lon2pix2.shape,lat2pix2.shape)
  # ang2pixでhealpixの番号に変換

  pixnumber=hp.pixelfunc.ang2pix(nside, lon2pix2, lat2pix2, nest=True, lonlat=True)
  #print(np.unique(pixnumber).shape) #healpixの分割数 #64bit

  return pixnumber

In [41]:
# imgからhealpix numberとそれぞれのdemデータの平均値を返す
def img2healpix(url):
  import re  #正規表現用
  import os
  import requests #データダウンロード用
  import tensorflow as tf #resampling用

  # urlをlbl→imgに変換
  url_img = re.sub('lbl','img',url)
  filename = os.path.basename(url_img)
  img = urllib.request.urlretrieve(url_img, f'{filename}')

  #ヘッダファイルの必要な要素を取り出す関数
  dictionary = lbl2dict(url)

  #リシェイプ
  data = np.fromfile(filename,dtype='>i2') #i2 int型変換
  data = data.reshape(dictionary["LINE"],dictionary["LINESAMPLE"])
  #data_mask = data.copy()

  #dataとpixnumberを対応させると高度の値が取れる

  #fig = plt.figure(figsize=(15,15))
  #ax = plt.gca()

  pixnum = readdics(url,nside)
  pixnum_unique= np.unique(readdics(url,nside))

  #リサンプリング
  healpix_dem=tf.sparse.bincount(np.ravel(pixnum), weights=np.ravel(data.astype(float)))
  healpix_cnt=tf.sparse.bincount(np.ravel(pixnum))
  healpix_cnt=tf.cast(healpix_cnt, tf.float64)
  ave = tf.cast(healpix_dem.values, dtype=tf.float64) / tf.cast(healpix_cnt.values, dtype=tf.float64)

  return pixnum_unique,ave


In [42]:
np.bincount(np.array([0, 1, 1, 3, 2, 1, 7,100]))


# 配列の長さがhealpixに含まれる画素数の最大値に依存する

array([1, 3, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [43]:
# 中平さんコード

import tensorflow as tf

pixnum = readdics(url, nside)
#pixnum_unique= np.unique( pixnum )

s=tf.sparse.bincount(np.ravel(pixnum), weights=np.ravel(data.astype(float)))
W=tf.sparse.bincount(np.ravel(pixnum))
w=tf.cast(W, tf.float64)
print(w.dtype)
print(s.dtype)
ans = s.values/w.values
print(ans)

#pixnumに値がある場合のみ計算する
#値が計算されたpix番号を取得する必要がある


#s=np.bincount(np.ravel(pixnum), weights=np.ravel(data.astype(float)), minlength=hp.nside2npix(nside) )
#w=np.bincount(np.ravel(pixnum), minlength=hp.nside2npix(nside)) #nsideに対応してpix数を数えて

<dtype: 'float64'>
<dtype: 'float64'>
tf.Tensor(
[-620.86473375 -598.50503625 -631.89556944 -663.95805889 -658.89737094
 -655.74672869 -664.54844868 -665.11298151 -601.52181501 -579.51511569
 -553.64983392 -575.29085888 -553.78744857 -566.02230585 -613.9753983
 -615.74815584 -597.58913148 -585.41032213 -596.16203192 -612.87200051
 -600.36083036 -614.95584016 -571.71760471 -582.79228168 -579.39509562
 -599.93052172 -603.09372841 -612.01087075 -597.30923468 -606.58774768
 -598.77475652 -602.5081551  -617.60395893 -615.86582196 -557.83602385
 -566.10225042 -581.71173038 -554.01577895 -643.64073152 -635.36305466
 -655.90729127 -637.73072169 -608.48285766 -574.18500273 -617.80661698
 -609.6902745  -588.6332485  -601.80469006 -607.30369024 -605.59790947
 -599.02201418 -611.71885396 -592.3015798  -607.67163555 -590.70212689
 -609.73308852 -600.46850401 -608.60879129 -606.10531004 -613.97207577
 -615.84785296 -598.44642597 -734.02651579 -693.44873085 -736.48767469
 -737.38937701 -693.76344086 

In [44]:
# tensorflow 勉強コード
import tensorflow as tf

pixnum = np.array([1, 1, 1, 1,1,2, 2, 2,2,2])
dem = np.array([6, 5, 6, 6, 6, 7,7,7,6,7])

s=tf.sparse.bincount(np.ravel(pixnum), weights=np.ravel(dem.astype(float)))
W=tf.sparse.bincount(np.ravel(pixnum))
W=tf.cast(W, tf.float64)
ans = s.values/W.values
print(s.values)
print(W.values)
print(ans)

tf.Tensor([29. 34.], shape=(2,), dtype=float64)
tf.Tensor([5. 5.], shape=(2,), dtype=float64)
tf.Tensor([5.8 6.8], shape=(2,), dtype=float64)


In [48]:
import numpy as np
from astropy.io import fits


healpix = img2healpix(url)

hdu = fits.PrimaryHDU(healpix[1])
hdulist = fits.HDUList()

hdulist[0] = healpix[0]
hdulist[1] = healpix[1]


hdu.writeto('make_fits_test.fits',overwrite=True)

In [ ]:
import astropy.io.fits as pyfits

import healpy as hp

import numpy as np

 

nside=4096*4

 
EXTNAME = 'SKYMAP  '   

INDXSCHM= 'EXPLICIT'                                                           

 

PIX=np.arange(196608,196608*2,1)

HEIGHT= np.random.uniform(0,10, len(PIX))

 

cols = pyfits.ColDefs([

  pyfits.Column(name='PIX', format='I',array=PIX),

  pyfits.Column(name='HEIGHT', format='D',array=HEIGHT) ])

 

a=pyfits.BinTableHDU.from_columns(cols)

a.header["NSIDE"]=nside

a.header["ORDERING"]="NESTED"

a.header["INDXSCHM"]="EXPLICIT"

a.header["PIXTYPE"]="HEALPIX"

a.header["FIRSTPIX"]=int(196608/2)

a.header["LASTPIX"]=int(196608/2)+196608

 

a.writeto("test.fits", overwrite=True)


# Hips変換
#java -Xmx64g -jar AladinBeta.jar -hipsgen in=test.fits out=PilotHiPS creator_did=HiPS hips_order=9

In [53]:
no = 9
ns = 2 ** 9
a= ns**2*12
print(a)

3145728


In [ ]:
    import astropy.io.fits as fits
    import matplotlib.pyplot as plt
    hdulist=pyfits.open(fits_url)
    hdu=hdulist[0]
    data=hdu.data #data=fits.getdata('file.fits',0)でもできる。
    header=hdu.header
    plt.imshow(data)
    plt.show()